## Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream 

## AMIGOS dataset

## RPWE


## Date: 11 Oct 2021 at 10:10 AM

In [2]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import tensorflow as tf
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from window_slider import Slider
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import SimpleRNN
from keras.utils import np_utils
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


# from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
# from skmultiflow.meta import AdaptiveRandomForest

from window_slider import Slider

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [3]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [4]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [5]:
#======================================================
# ECG Feature Extraction (Wavelet Features)
#======================================================

def extract_ecg_features(raw_data):
    ch =0
    features =[]
    
    def get_features(list_values):    
        list_values = list_values
        entropy = calculate_entropy(list_values)
        crossings = calculate_crossings(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + crossings + statistics

    for i in range(raw_data.shape[0]):
            ch_data = raw_data[i]
            list_coeff = pywt.wavedec(ch_data, 'db4', level=3)
            for coeff in list_coeff:
                features += get_features(coeff)
            ch = ch+1

    return features

In [6]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [7]:
def create_model(x):
    learning_rate = 0.05
    sgd = tf.keras.optimizers.SGD(lr=learning_rate)
    dim = x.shape[1]
    model = Sequential()
    model.add(Input(shape=(dim)))
    model.add(Dense(math.ceil((2/3)*dim),input_dim=dim,activation='sigmoid'))
    model.add(Dense(1,activation='sigmoid'))
    print(model.summary())
    model.compile(optimizer=sgd,
                loss='mse',
                metrics=['accuracy'])
    return model

In [8]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((7085/51)*segment_in_sec)  #7085 is for 51 sec video record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers


# optimizer= 'SGD' #optimizer
# classifier = 'MLP_RPWE'+str(optimizer)
indx = 0
c =0
ccc = 0

participant = 40
videos = 16

global eeg_emotion, eda_emotion, ecg_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
ecg_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()


eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()


ecg_acc_val = metrics.Accuracy() #Accuracy
ecg_f1m_val = metrics.F1() #F1 measure  
ecg_acc_aro = metrics.Accuracy() #Accuracy
ecg_f1m_aro = metrics.F1() #F1 measure
ecg_roc_val = metrics.ROCAUC()
ecg_roc_aro = metrics.ROCAUC()
ecg_mcc_val = metrics.MCC()
ecg_mcc_aro = metrics.MCC()

val_MSE = []
aro_MSE = []

haha_val = []
haha_aro = []

all_weights_aro = []
all_weights_val = []


 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
ecg_cm_val  = metrics.ConfusionMatrix()
ecg_cm_aro  = metrics.ConfusionMatrix()


    
itr = 0 #controls the learning rate


#=======================================
# MAIN Loop STARTS HERE
#=======================================
for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/'+str(p)+'data_eeg_AMIGOS'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/'+str(p)+'data_eda_AMIGOS.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # ECG data read from files
        ##===================================================
        def ecg_data(p):
            file_ecg = '/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/'+str(p)+'data_ecg_AMIGOS.csv'
            print(file_ecg)
            ecg_sig = pd.read_csv(file_ecg,sep=',', header = None, engine='python')
            return ecg_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_ecg = ecg_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            ecg_sig = grand_ecg.loc[grand_ecg.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            aro = eeg_sig.iloc[0,7087]
            val = eeg_sig.iloc[0,7088]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al]) 
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            ecg_val_prdt=[]
            ecg_aro_prdt =[]
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_ecg = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,14),range(2,7087)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(2,7087)]) #keeping only eda signals
            ecg_sig = np.array(ecg_sig.iloc[range(0,2),range(2,7087)]) #keeping only ecg signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_ecg.fit(ecg_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_ecg = slider_ecg.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                window_data_eeg  = np.nan_to_num(window_data_eeg) #Replacing NaN with 0 if there is such case
                features_eeg = eeg_features(window_data_eeg)
                eeg = np.array([features_eeg])  #EEG raw feature vector
                x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                window_data_eda  = np.nan_to_num(window_data_eda) #Replacing NaN with 0 if there is such case
                eda_features = extract_eda_features(np.array(window_data_eda))
                eda = np.array([eda_features]) #EDA raw feature vector
                x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from ecg belt
                #=================================================
                window_data_ecg  = np.nan_to_num(window_data_ecg) #Replacing NaN with 0 if there is such case
                ecg_features = extract_ecg_features(np.array(window_data_ecg))
                ecg = np.array([ecg_features]) #ecg BELT raw feature vector
                x_ecg = preprocessing.normalize(ecg) #ecg BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('ecg BELT Feature shape{}:'.format(x_ecg.shape))

                    #========================
                    # For EEG data model
                    #========================
                    eeg_model_val = create_model(x_eeg)

                    eeg_model_aro = create_model(x_eeg)

                    #========================
                    # For EDA data model
                    #========================
                    eda_model_val = create_model(x_eda)

                    eda_model_aro = create_model(x_eda)

                    #==============================
                    # For ecg Belt data Model
                    #==============================
                    ecg_model_val = create_model(x_ecg)

                    ecg_model_aro = create_model(x_ecg)

                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                if c == 0: #For the first time model will return 0 or None
                    tmp_eeg_val = 0
                    tmp_eeg_aro = 0
                    
                    tmp_eda_val = 0
                    tmp_eda_aro = 0
                    
                    tmp_ecg_val = 0
                    tmp_ecg_aro = 0
                    
                    
                    y_pred_val_eeg = [2]
                    y_pred_aro_eeg = [2]
                    
                    y_pred_val_eda = [2]
                    y_pred_aro_eda = [2]
                    
                    y_pred_val_ecg = [2]
                    y_pred_aro_ecg = [2]
                    
                    eeg_model_val.fit(np.array([x_eeg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    ecg_model_val.fit(np.array([x_ecg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    ecg_model_aro.fit(np.array([x_ecg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    c=c+1
                    
                else: 
                    
                    tmp_eeg_val  = eeg_model_val.predict(np.array([x_eeg]))[0][0][0]
                    tmp_eeg_aro = eeg_model_aro.predict(np.array([x_eeg]))[0][0][0]
                    
                    tmp_eda_val = eda_model_val.predict(np.array([x_eda]))[0][0][0]
                    tmp_eda_aro  = eda_model_aro.predict(np.array([x_eda]))[0][0][0]
                    
                    tmp_ecg_val = ecg_model_val.predict(np.array([x_ecg]))[0][0][0]
                    tmp_ecg_aro = ecg_model_aro.predict(np.array([x_ecg]))[0][0][0]
                    
                    #Binary output scaling 1 if x > 0.5 else 0
                    y_pred_val_eeg = [1 if tmp_eeg_val > 0.5 else 0]
                    y_pred_aro_eeg = [1 if tmp_eeg_aro > 0.5 else 0]
                    
                    y_pred_val_eda = [1 if tmp_eda_val > 0.5 else 0]
                    y_pred_aro_eda = [1 if tmp_eda_aro > 0.5 else 0]
                    
                    y_pred_val_ecg = [1 if tmp_ecg_val > 0.5 else 0]
                    y_pred_aro_ecg = [1 if tmp_ecg_aro > 0.5 else 0]
                    
                    #Fit the model once
                    eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    ecg_model_val.fit(np.array([x_ecg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    ecg_model_aro.fit(np.array([x_ecg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    

                #===========================================
                # Performance matric update
                #===========================================
                
                #Valence classification EEG

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg[0])  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg[0]) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                #Arousal classification EEG 

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg[0]) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])


                #Valence classification EDA

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda[0])  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda[0]) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda[0])
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda[0])
                

                #Arousal classification EDA
                
                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda[0])  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda[0]) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda[0])
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

                #Valence classification ecg Belt

                ecg_acc_val = ecg_acc_val.update(y_act_val[0], y_pred_val_ecg[0])  # update the accuracy metric

                ecg_f1m_val = ecg_f1m_val.update(y_act_val[0], y_pred_val_ecg[0]) #update f1 measure metric
                
                ecg_roc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg[0])
                
                ecg_mcc_val = ecg_roc_val.update(y_act_val[0], y_pred_val_ecg[0])
                

                #Arousal classification ecg Belt

                ecg_acc_aro = ecg_acc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])  # update the accuracy metric

                ecg_f1m_aro = ecg_f1m_aro.update(y_act_aro[0], y_pred_aro_ecg[0]) #update f1 measure metric
                
                ecg_roc_aro = ecg_roc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])
                
                ecg_mcc_aro = ecg_mcc_aro.update(y_act_aro[0], y_pred_aro_ecg[0])
                

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))
            
            ecg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_ecg_val))
            ecg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_ecg_aro))
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, ecg_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, ecg_aro_MSE] #Arousal errors
            

            
            #=================================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #=================================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
                
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
            
            #============================================
            # Controls for the first time
            #============================================
            if ccc ==0:
                val_label = [2, 2, 2]
                aro_label = [2, 2, 2] 
                ccc =ccc+1
            else:
                val_label = [tmp_eeg_val, tmp_eda_val, tmp_ecg_val]
                aro_label = [tmp_eeg_aro, tmp_eda_aro, tmp_ecg_aro] 
            
            
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg[0])
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda[0])
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda[0])
            
            ecg_cm_val = ecg_cm_val.update(y_act_val[0], y_pred_val_ecg[0])
            ecg_cm_varo = ecg_cm_aro.update(y_act_aro[0], y_pred_aro_ecg[0])            
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot(val_label, w_val)
            mer_val = 1 if p_val > 0.5 else 0
            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_aro = np.dot(aro_label,w_aro)
            mer_aro = 1 if p_aro > 0.5 else 0
            
            print([val_label, aro_label])
            print([p_val,p_aro])
            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling
            
            #============================================
            # Weight update for Arousal
            #============================================
            
            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            all_weights_val.append(w_val)
            all_weights_aro.append(w_aro)
            
#             mer_val = val_label[np.argmax(w_val)] #Final Class prediction Valence
#             mer_aro = aro_label[np.argmax(w_aro)] #Final Class Prediction Valence
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val  = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro  = mer_cm_aro.update(y_act_aro[0], mer_aro) 
            
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                        eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                        y_act_val[0], y_pred_val_eeg[0], y_act_aro[0], y_pred_aro_eeg[0]]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                        eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                        y_act_val[0], y_pred_val_eda[0], y_act_aro[0], y_pred_aro_eda[0]]))
            
            ecg_emotion.append(np.array([p,v, ecg_acc_val.get(), ecg_f1m_val.get(), ecg_roc_val.get(), ecg_mcc_val.get(),
                                          ecg_acc_aro.get(),ecg_f1m_aro.get(), ecg_roc_aro.get(), ecg_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_ecg[0], y_act_aro[0], y_pred_aro_ecg[0]]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                        mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                        y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/AMIGOS_data/eeg_data/1data_eeg_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/eda_data/1data_eda_AMIGOS.csv
/home/gp/Desktop/MER_arin/AMIGOS_data/ecg_data/1data_ecg_AMIGOS.csv
Person: 1 Video:1
EEG Feature shape(1, 504):
EDA Feature shape(1, 48):
ecg BELT Feature shape(1, 96):
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 336)               169680    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 337       
Total params: 170,017
Trainable params: 170,017
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (ty

[[2, 2, 2], [2, 2, 2]]
[2.0, 2.0]
MER model
Valence Accuracy:0.0
Valence F1 score:0.0
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:0.0
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:2
[[0.844885, 0.5407074, 0.5819391], [0.07797241, 0.20755509, 0.2811806]]
[0.6558438340822855, 0.18890269597371417]
MER model
Valence Accuracy:0.5
Valence F1 score:0.6667
Valence ROC score:0.5
Valence MCC score:0.0
Arousal Accuracy:0.5
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:3
[[0.14970875, 0.4294512, 0.36403123], [0.06340936, 0.18131891, 0.19778183]]
[0.3139242719520389, 0.1475033660729726]
MER model
Valence Accuracy:0.6667
Valence F1 score:0.6667
Valence ROC score:0.75
Valence MCC score:0.5
Arousal Accuracy:0.6667
Arousal F1 score:0.0
Arousal ROC score:-0.0
Arousal MCC score:0.0


[[0.06566054, 0.21410763, 0.1579949], [0.91269535, 0.6756221, 0.7776336]]
[0.14363094134170643, 0.8004582961286367]
MER model
Valence Accuracy:0.9091
Valence F1 score:0.8571
Valence ROC score:0.9375
Valence MCC score:0.8101
Arousal Accuracy:0.8182
Arousal F1 score:0.8
Arousal ROC score:0.8167
Arousal MCC score:0.6333
===============================xxxx================================
Person: 1 Video:12
[[0.7918778, 0.33183295, 0.3587678], [0.17530519, 0.5318264, 0.5246761]]
[0.5036307257514199, 0.3903811903843195]
MER model
Valence Accuracy:0.9167
Valence F1 score:0.8889
Valence ROC score:0.9375
Valence MCC score:0.8367
Arousal Accuracy:0.8333
Arousal F1 score:0.8
Arousal ROC score:0.8286
Arousal MCC score:0.6571
===============================xxxx================================
Person: 1 Video:13
[[0.9026674, 0.51468927, 0.6577146], [0.8549655, 0.6250473, 0.6745586]]
[0.70392759667015, 0.7332315144403527]
MER model
Valence Accuracy:0.9231
Valence F1 score:0.9091
Valence ROC score:0.9

[[0.037882924, 0.16199523, 0.09774956], [0.9022253, 0.44489256, 0.6071869]]
[0.08950652985040383, 0.7143525210002672]
MER model
Valence Accuracy:0.9286
Valence F1 score:0.9167
Valence ROC score:0.9412
Valence MCC score:0.8641
Arousal Accuracy:0.9286
Arousal F1 score:0.9
Arousal ROC score:0.9222
Arousal MCC score:0.8444
===============================xxxx================================
Person: 2 Video:13
[[0.46213806, 0.2410976, 0.19638795], [0.9280361, 0.6012248, 0.7660172]]
[0.3237801983775607, 0.8103954306467651]
MER model
Valence Accuracy:0.8966
Valence F1 score:0.88
Valence ROC score:0.8995
Valence MCC score:0.7913
Arousal Accuracy:0.931
Arousal F1 score:0.9091
Arousal ROC score:0.9268
Arousal MCC score:0.8535
===============================xxxx================================
Person: 2 Video:14
[[0.890844, 0.40022993, 0.51070046], [0.19539073, 0.46949238, 0.5100827]]
[0.651728563636144, 0.3414531389658921]
MER model
Valence Accuracy:0.9
Valence F1 score:0.8889
Valence ROC score:0

[[0.45687968, 0.22448099, 0.19683993], [0.8575697, 0.5934118, 0.6734148]]
[0.3376110709946617, 0.7680645862010252]
MER model
Valence Accuracy:0.8889
Valence F1 score:0.8718
Valence ROC score:0.8897
Valence MCC score:0.7747
Arousal Accuracy:0.9333
Arousal F1 score:0.9032
Arousal ROC score:0.9203
Arousal MCC score:0.8535
===============================xxxx================================
Person: 3 Video:14
[[0.8902032, 0.38744846, 0.5035328], [0.15019271, 0.4606721, 0.42188108]]
[0.6848450905366023, 0.2666188750298784]
MER model
Valence Accuracy:0.8913
Valence F1 score:0.878
Valence ROC score:0.8923
Valence MCC score:0.7809
Arousal Accuracy:0.9348
Arousal F1 score:0.9032
Arousal ROC score:0.9208
Arousal MCC score:0.8551
===============================xxxx================================
Person: 3 Video:15
[[0.9255271, 0.5604806, 0.71957755], [0.84611857, 0.57789147, 0.62940735]]
[0.7996985630898382, 0.7516161521782556]
MER model
Valence Accuracy:0.8936
Valence F1 score:0.8837
Valence ROC

...

[[0.08567631, 0.2818359, 0.2063069], [0.08539948, 0.24912098, 0.19069934]]
[0.08567894248535944, 0.08541569095358825]
MER model
Valence Accuracy:0.9667
Valence F1 score:0.9654
Valence ROC score:0.9665
Valence MCC score:0.9333
Arousal Accuracy:0.9719
Arousal F1 score:0.9738
Arousal ROC score:0.9712
Arousal MCC score:0.9438
===============================xxxx================================
Person: 36 Video:11
[[0.06685883, 0.23192915, 0.16070563], [0.81978905, 0.38111395, 0.43027312]]
[0.06686088798051427, 0.8197332627460107]
MER model
Valence Accuracy:0.9667
Valence F1 score:0.9654
Valence ROC score:0.9665
Valence MCC score:0.9334
Arousal Accuracy:0.972
Arousal F1 score:0.9739
Arousal ROC score:0.9712
Arousal MCC score:0.9439
===============================xxxx================================
Person: 36 Video:12
[[0.056337595, 0.19909066, 0.13241506], [0.14534059, 0.32159102, 0.28319043]]
[0.05633927946976861, 0.14535958880964006]
MER model
Valence Accuracy:0.9668
Valence F1 score:0.96

[[0.041072905, 0.16823193, 0.10449809], [0.853425, 0.42834944, 0.56165427]]
[0.041074007910063805, 0.853394001151606]
MER model
Valence Accuracy:0.9676
Valence F1 score:0.966
Valence ROC score:0.9674
Valence MCC score:0.9352
Arousal Accuracy:0.971
Arousal F1 score:0.9726
Arousal ROC score:0.9705
Arousal MCC score:0.942
===============================xxxx================================
Person: 37 Video:12
[[0.037386417, 0.1428999, 0.09651518], [0.1521863, 0.3617497, 0.363945]]
[0.03738742776510649, 0.15220636177891764]
MER model
Valence Accuracy:0.9677
Valence F1 score:0.966
Valence ROC score:0.9674
Valence MCC score:0.9353
Arousal Accuracy:0.9711
Arousal F1 score:0.9726
Arousal ROC score:0.9706
Arousal MCC score:0.9421
===============================xxxx================================
Person: 37 Video:13
[[0.46287316, 0.21369702, 0.18863016], [0.8452274, 0.49879128, 0.6017928]]
[0.4628688020650139, 0.8452032122836681]
MER model
Valence Accuracy:0.966
Valence F1 score:0.9643
Valence R

[[0.53427386, 0.23477164, 0.21155286], [0.9508443, 0.7971395, 0.86900383]]
[0.5342696924899334, 0.9508380684435767]
MER model
Valence Accuracy:0.9669
Valence F1 score:0.965
Valence ROC score:0.9665
Valence MCC score:0.9337
Arousal Accuracy:0.9719
Arousal F1 score:0.9734
Arousal ROC score:0.9714
Arousal MCC score:0.9437
===============================xxxx================================
Person: 38 Video:13
[[0.89476943, 0.3956567, 0.5284721], [0.95353496, 0.8219151, 0.8852021]]
[0.8947649322918272, 0.9535297338975511]
MER model
Valence Accuracy:0.9669
Valence F1 score:0.9652
Valence ROC score:0.9666
Valence MCC score:0.9338
Arousal Accuracy:0.9719
Arousal F1 score:0.9735
Arousal ROC score:0.9714
Arousal MCC score:0.9437
===============================xxxx================================
Person: 38 Video:14
[[0.92407954, 0.5638294, 0.7277143], [0.34700078, 0.72266304, 0.7503342]]
[0.92407715363536, 0.34702748406385736]
MER model
Valence Accuracy:0.967
Valence F1 score:0.9653
Valence ROC 

[[0.894537, 0.4258747, 0.54368615], [0.9566966, 0.8179811, 0.87948895]]
[0.8945338851306754, 0.9566924988133036]
MER model
Valence Accuracy:0.9678
Valence F1 score:0.9661
Valence ROC score:0.9675
Valence MCC score:0.9355
Arousal Accuracy:0.9726
Arousal F1 score:0.9741
Arousal ROC score:0.9722
Arousal MCC score:0.9452
===============================xxxx================================
Person: 39 Video:14
[[0.9245591, 0.5850352, 0.7359024], [0.41492325, 0.72391605, 0.75066084]]
[0.9245574745641002, 0.4149391137837241]
MER model
Valence Accuracy:0.9678
Valence F1 score:0.9662
Valence ROC score:0.9675
Valence MCC score:0.9356
Arousal Accuracy:0.9727
Arousal F1 score:0.9741
Arousal ROC score:0.9722
Arousal MCC score:0.9453
===============================xxxx================================
Person: 39 Video:15
[[0.9389237, 0.687026, 0.805216], [0.10439438, 0.5445155, 0.41596404]]
[0.9389225624019204, 0.10440893515370131]
MER model
Valence Accuracy:0.9679
Valence F1 score:0.9663
Valence ROC s

[[0.92824113, 0.64754355, 0.77009326], [0.3612635, 0.72142184, 0.74092185]]
[0.9282400786486829, 0.3612769863505907]
MER model
Valence Accuracy:0.9687
Valence F1 score:0.9671
Valence ROC score:0.9683
Valence MCC score:0.9372
Arousal Accuracy:0.9734
Arousal F1 score:0.9748
Arousal ROC score:0.9729
Arousal MCC score:0.9466
===============================xxxx================================
Person: 40 Video:15
[[0.9408294, 0.72614646, 0.82579464], [0.8654974, 0.7405099, 0.78116435]]
[0.9408286297138322, 0.8654944594247325]
MER model
Valence Accuracy:0.9687
Valence F1 score:0.9672
Valence ROC score:0.9684
Valence MCC score:0.9373
Arousal Accuracy:0.9734
Arousal F1 score:0.9749
Arousal ROC score:0.9729
Arousal MCC score:0.9467
===============================xxxx================================
Person: 40 Video:16
[[0.94901466, 0.77084446, 0.85354006], [0.9163202, 0.78261375, 0.8229351]]
[0.9490140283338406, 0.9163169708548662]
MER model
Valence Accuracy:0.9687
Valence F1 score:0.9673
Valenc

In [9]:
all_weights_aro = pd.DataFrame(all_weights_aro)
all_weights_val = pd.DataFrame(all_weights_val)
all_weights_aro.to_csv('all_weights_aro_AMIGOS.csv')
all_weights_val.to_csv('all_weights_val_AMIGOS.csv')

In [15]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('ecg Val CM:')
print(ecg_cm_val)
print('ecg Aro CM:')
print(ecg_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    330      2
      1     13    295
EEG aro CM:
             0      1
      0    294      9
      1      5    332
EDA Val CM:
             0      1
      0    262     70
      1     84    224
EDA aro CM:
             0      1
      0    228     75
      1     71    266
ecg Val CM:
             0      1
      0    274     58
      1     53    255
ecg Aro CM:
             0      1
      0    242     61
      1     48    289
ReMECS Val CM:
             0      1
      0    324      8
      1     13    295
ReMECS aro CM:
             0      1
      0    292     11
      1      6    331


In [17]:
#======================================
# Result Save For each Classifier
#======================================
classifier = 'RPWE'
filename = "/home/gp/Desktop/MER_arin/mer_codes/Exp-2/AMIGOS/Final_Results/"
fname_eeg = filename + 'AMIGOS_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'AMIGOS_all_person_EDA'+'_' +classifier+'_results.csv'
fname_ecg = filename + 'AMIGOS_all_person_ECG'+'_' +classifier+'_results.csv'
fname_mer = filename + 'AMIGOS_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
ecg_emotion = pd.DataFrame(ecg_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
ecg_emotion.to_csv(fname_ecg)
mer_emotion.to_csv(fname_mer)